# Se carga inicialmente el SUBSET de 714 audios (con padding) de los 59139 audios de entrenamiento

In [1]:
import os
import glob
import numpy as np
import pandas as pd


# Cargar el DataFrame de entrenamiento pequeños (subset)
train_subset = pd.read_csv("subset_train_data.csv")

# Extraer la columna 'file name' como una lista
wav_files_list = train_subset["file name"].tolist()# Crear la lista 'annotations' con la ruta y la extensión modificada

# Obtener todas las rutas de archivos dentro de train/ con sus subdirectorios
# wav_files_paths = glob.glob("/teamspace/studios/single-classification/data/train/*/*.WAV", recursive=True)
wav_files_paths = glob.glob("/teamspace/studios/opensoundscape/extended_audios/train/*/*.WAV", recursive=True)


# Crear un diccionario donde la clave es el nombre del archivo y el valor es su ruta completa. 
# Contiene toda la lista de audios de entrenamiento, no solo los del subset.
wav_files_dict = {os.path.basename(f): f for f in wav_files_paths}

# Asegurar que las rutas sean correctas buscando en el diccionario
audios = [wav_files_dict[file + ".WAV"] for file in wav_files_list if (file + ".WAV") in wav_files_dict]

# Verificar cuántos archivos se encontraron correctamente
print(f"Archivos encontrados: {len(audios)}/{len(wav_files_list)}")
print(audios[:5])  # Verificar las primeras rutas generadas
print(f"Total de audios con cantos en todo el dataset: {len(wav_files_paths)}")

Archivos encontrados: 714/714
['/teamspace/studios/opensoundscape/extended_audios/train/pf71436/20221W60_20220419_050000_105690240.WAV', '/teamspace/studios/opensoundscape/extended_audios/train/pf71436/20221W60_20220419_040000_151281792.WAV', '/teamspace/studios/opensoundscape/extended_audios/train/pf71436/20221W60_20220419_040000_148187520.WAV', '/teamspace/studios/opensoundscape/extended_audios/train/pf71436/20221W60_20220419_040000_150959232.WAV', '/teamspace/studios/opensoundscape/extended_audios/train/pf71436/20221W60_20220419_040000_149819904.WAV']
Total de audios con cantos en todo el dataset: 59139


# Se utiliza birdnet lib para extraer Embeddings de los 714 audios con padding.

In [2]:
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
import pandas as pd
import os

# Inicializar el modelo BirdNET
analyzer = Analyzer()

# Lista de rutas de los archivos de audio
# Asegúrate de que 'audios' ya está definida y contiene las rutas correctas
all_data = []

# Recorrer cada archivo en la lista de audios
for audio_path in audios:
    try:
        # Extraer el nombre del archivo sin la ruta
        file_name = os.path.basename(audio_path)

        # Cargar la grabación en BirdNET
        recording = Recording(analyzer, audio_path)
        recording.extract_embeddings()

        # Procesar los embeddings extraídos
        for emb in recording.embeddings:
            row = {
                "path": audio_path,  # Ruta completa del archivo
                "file_name": file_name,  # Nombre del archivo sin la ruta
                "start_time": emb['start_time'],
                "end_time": emb['end_time']
            }
            # Añadir los 1024 valores del embedding
            for i, value in enumerate(emb["embeddings"]):
                row[f"dim_{i+1}"] = value

            all_data.append(row)

    except Exception as e:
        print(f"Error procesando {audio_path}: {e}")

# Crear DataFrame final con todos los embeddings
df_embeddings = pd.DataFrame(all_data).set_index("file_name")

# Mostrar el DataFrame
# import ace_tools as tools
# tools.display_dataframe_to_user(name="All Embeddings DataFrame", dataframe=df_embeddings)


Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.
read_audio_data


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 20221W60_20220419_050000_105690240.WAV
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 20221W60_20220419_040000_151281792.WAV
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 20221W60_20220419_040000_148187520.WAV
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 20221W60_20220419_040000_150959232.WAV
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 20221W60_20220419_040000_149819904.WAV
read_audio_data
read_audio_data: complete, read  2 chunks.
extract_embeddings_for_recording 20221W60_20220419_050000_50821120.WAV
read_audio_data
read_audio_data: complete, read  1 chunks.
extract_embeddings_for_recording 20221W60_20220419_040000_148452864.WAV
read_audio_data
read_audio_data: complete, read  2 chunks.
extract_embeddings_for_recording 20221W60

# Contar el número de audios procesados para ver si todos quedaron con embeddings.

In [3]:
num_audios_procesados = df_embeddings["path"].nunique()  # O el nombre correcto de la columna
print(f"Se lograron procesar {num_audios_procesados} audios diferentes.")

# Convertir la lista original de audios a un conjunto
audios_originales = set(audios)

# Obtener la lista de audios procesados del DataFrame
audios_procesados = set(df_embeddings["path"].unique())

# Identificar los audios que no se procesaron
audios_no_procesados = audios_originales - audios_procesados

# Mostrar resultados
print(f"Total de audios en la lista original: {len(audios_originales)}")
print(f"Total de audios procesados: {len(audios_procesados)}")
print(f"Total de audios no procesados: {len(audios_no_procesados)}")

# Si quieres ver qué audios no se procesaron:
print("Audios no procesados:")
print("\n".join(audios_no_procesados))


Se lograron procesar 714 audios diferentes.
Total de audios en la lista original: 714
Total de audios procesados: 714
Total de audios no procesados: 0
Audios no procesados:



In [ ]:
# Guardar el DataFrame en un archivo CSV
csv_filename = "embeddings_train_with_padding_subset.csv"
df_embeddings.to_csv(csv_filename, index=True)

print(f"Embeddings guardados en {csv_filename}")

Embeddings guardados en embeddings_train_with_padding.csv
